In [1]:
import pandas as pd
import textwrap
import plotly.graph_objs as go
import plotly.io as pio
from pathlib import Path #To create unique filenames for each PFA chart
import itertools

import src.data.utilities as utils
import src.visualization.prt_theme as prt_theme

config = utils.read_config()
pio.templates.default = "prt_template"

In [2]:
def loadData(status='processed', filename='PFA_2022_offences.csv') -> pd.DataFrame:
    """Load CSV file into Pandas DataFrame and convert object columns to categories when they meet criteria in `categoryColumns()`

    Parameters
    ----------
    status : {'raw', 'interim', 'processed'}, default is 'interim'
        Status of the data processing.
        * If 'raw' file is located in "rawFilePath" within config file
        * If 'interim', file is located in "intFilePath"
        * If 'processed', file is located in "clnFilePath"
    filename : str, default is 'PFA_2010-22_women_cust_comm_sus.csv'
        Name of CSV file to be loaded.

    Returns
    -------
    DataFrame
        CSV data is returned as Pandas DataFrame with any eligible object columns converted into category columns to limit memory requirements
    """
    paths = {
        "raw": 'rawFilePath',
        "interim": 'intFilePath',
        "processed": 'clnFilePath'
    }

    dfPath=f"{config['data'][paths[status]]}{filename}"
    df = pd.read_csv(dfPath)
    print('Data loaded')
    return utils.categoryColumns(df)

In [3]:
df = loadData()

Data loaded


In [4]:
df.columns

Index(['pfa', 'Criminal damage and arson', 'Drug offences', 'Fraud Offences',
       'Miscellaneous crimes against society', 'Possession of weapons',
       'Public order offences', 'Robbery', 'Sexual offences',
       'Summary motoring', 'Summary non-motoring', 'Theft offences',
       'Violence against the person'],
      dtype='object')

Fraud offences has a capital letter, let's remove. I'll circle back and resolve this in the processing code later on.

In [5]:
df = df.rename({'Fraud Offences': 'Fraud offences'}, axis=1)

In [6]:
#Melting df2 from wide to long
df2 = pd.melt(df, id_vars='pfa', value_vars=df.columns[1:], var_name='offence', value_name='proportion')

#Selecting the offences that I want to continue to display at the root of the sunburst diagram
highlighted_offence_groups = ['Theft offences', 'Drug offences', 'Violence against the person']
filt = df2['offence'].isin(highlighted_offence_groups)

df2.loc[filt, 'parent'] = "All offences" # This method prevents that annoying copy/view warning
df2.loc[~filt, 'parent'] = "All other<br>offences"

#Setting discreet plotting order
plot_dict = {
    'All other offences': 0,
    'Theft offences': 1,
    'Drug offences': 2,
    'Violence against the person': 3
}

df2['plot_order'] = df2["offence"].map(plot_dict).fillna(0)

#Adding text wrapping for longer offence categories
def customwrap(s,width=19):
    return "<br>".join(textwrap.wrap(s,width=width))

df2['offence'] = df2['offence'].map(customwrap)


In [112]:
#Melting df2 from wide to long
df2 = pd.melt(df, id_vars='pfa', value_vars=df.columns[1:], var_name='offence', value_name='proportion')

#Selecting the offences that I want to continue to display at the root of the sunburst diagram
highlighted_offence_groups = ['Theft offences', 'Drug offences', 'Violence against the person']
filt = df2['offence'].isin(highlighted_offence_groups)

df2.loc[filt, 'parent'] = "All offences" # This method prevents that annoying copy/view warning
df2.loc[~filt, 'parent'] = "All other<br>offences"

#Adding discreet colour mapping labels

colors = prt_theme.pio.templates["prt_template"].layout.colorway

color_dict = {
    'All other offences': colors[0],
    'Theft offences': colors[1],
    'Drug offences': colors[2],
    'Violence against the person': colors[3]
}

df2["colors"] = df2["offence"].map(color_dict).fillna(colors[0])

#Adding text wrapping for longer offence categories
def customwrap(s,width=19):
    return "<br>".join(textwrap.wrap(s,width=width))

df2['offence'] = df2['offence'].map(customwrap)

In [147]:
#Setting discreet plotting order
plot_dict = {
    'All other offences': 0,
    'Theft offences': 1,
    'Drug offences': 2,
    'Violence against the person': 3
}

df2['plot_order'] = df2["offence"].map(plot_dict).fillna(0)

In [7]:
plot_order

NameError: name 'plot_order' is not defined

In [8]:
df2

,pfa,offence,proportion,parent,plot_order
0,Avon and Somerset,Criminal damage and<br>arson,0.017,All other<br>offences,0.0
1,Bedfordshire,Criminal damage and<br>arson,0.000,All other<br>offences,0.0
2,Cambridgeshire,Criminal damage and<br>arson,0.000,All other<br>offences,0.0
3,Cheshire,Criminal damage and<br>arson,0.014,All other<br>offences,0.0
4,Cleveland,Criminal damage and<br>arson,0.010,All other<br>offences,0.0
...,...,...,...,...,...
499,Warwickshire,Violence against<br>the person,0.220,All offences,3.0
500,West Mercia,Violence against<br>the person,0.255,All offences,3.0
501,West Midlands,Violence against<br>the person,0.231,All offences,3.0
502,West Yorkshire,Violence against<br>the person,0.195,All offences,3.0


In [149]:
def customwrap(s,width=19):
    return "<br>".join(textwrap.wrap(s,width=width))

df2['offence'] = df2['offence'].map(customwrap)

In [150]:
df2['offence'].unique()

array(['Criminal damage and<br>arson', 'Drug offences', 'Fraud offences',
       'Miscellaneous<br>crimes against<br>society',
       'Possession of<br>weapons', 'Public order<br>offences', 'Robbery',
       'Sexual offences', 'Summary motoring', 'Summary non-<br>motoring',
       'Theft offences', 'Violence against<br>the person'], dtype=object)

In [ ]:
#Filtering dataframe to remove values with null values
mask = pfa_df['proportion'] > 0
pfa_df = pfa_df[mask]

In [9]:
for pfa in df2['pfa'].unique():
    pfa_df = df2[df2["pfa"] == pfa]
    
    #Creating an 'All other offences' parent value to contain offences listed in highlighted_offence_groups
    pfa_df = pd.concat([pfa_df, 
                    pd.DataFrame.from_records([{'pfa': pfa_df['pfa'].iloc[0],
                                                'offence':"All other<br>offences", 
                                                'proportion': pfa_df.loc[~filt, 'proportion'].sum(), 
                                                'parent':"All offences"}])
                                                ],
                                                ignore_index=True
                    ).sort_values(by=['offence'], ascending=False)
    
    ## Chart title
    title = textwrap.wrap(f'<b>Imprisonment of women in {pfa_df["pfa"].iloc[0]}<br>by offence group in 2022</b>', width=100)

    fig = go.Figure()    

    fig.add_trace(go.Sunburst(
        labels=pfa_df['offence'],
        parents=pfa_df['parent'],
        values=pfa_df['proportion'],
        sort=False,
        branchvalues='total',
        texttemplate="%{label}<br>%{percentRoot: .0%}",
        hovertemplate="<b>%{label}</b><br>%{percentParent: .0%} of %{parent}<extra></extra>",
        hoverinfo='label+percent parent',
        insidetextorientation='horizontal',
        rotation=90,
    )
        )
    
    fig.update_layout(
    # margin = dict(t=80, l=0, r=0, b=50),
    title="<br>".join(title),
    title_y=0.94,
    title_yanchor="bottom",
    uniformtext_minsize=7,
    uniformtext_mode='hide',
    xaxis=dict(
        range=[-1, 1], 
        showticklabels=False
    ),
    yaxis=dict(
        range=[-1, 1],
        showticklabels=False
    ),
    width=600,
    height=600,
    margin=dict(t=75, b=20, r=0, l=0)
    )

    ## Chart annotations
    annotations = []
    # Adding source label
    source = prt_theme.sourceAnnotation("Ministry of Justice, Criminal justice statistics", annotations)

    # Adding annotations to layout
    fig.update_layout(annotations=annotations)

    fig.show()

    # export_path = Path.joinpath(Path.cwd(), f"{config['data']['outPath']}", "custody_offences_2022")
    # export_path.mkdir(parents=True, exist_ok=True) #generate if does not exist

    # Setting filename variable and full path

    # filename = str(pfa_df["pfa"].iloc[0])
    # export_eps_path = Path.joinpath(export_path, f'{filename}' + '.eps')

    # fig.write_image(export_eps_path)

In [ ]:
fig.data[0].values == 0.

Now with two charts

In [ ]:
pfa_df

In [32]:
prt_theme.pio.templates["prt_template"].layout.colorway

('#A01D28', '#499CC9', '#F9A237', '#6FBA3A', '#573D6B')

In [33]:
pfa_df

,pfa,offence,proportion,parent
11,Kent,Violence against the<br>person,0.224,All offences
10,Kent,Theft offences,0.385,All offences
9,Kent,Summary non-motoring,0.077,All other<br>offences
8,Kent,Summary motoring,0.014,All other<br>offences
7,Kent,Sexual offences,0.007,All other<br>offences
6,Kent,Robbery,0.021,All other<br>offences
5,Kent,Public order<br>offences,0.042,All other<br>offences
4,Kent,Possession of<br>weapons,0.007,All other<br>offences
3,Kent,Miscellaneous crimes<br>against society,0.091,All other<br>offences
2,Kent,Fraud offences,0.056,All other<br>offences


In [164]:
pfa_df = df2[df2["pfa"] == "Kent"]
pfa_df

,pfa,offence,proportion,parent,plot_order
18,Kent,Criminal damage and<br>arson,0.014,All other<br>offences,0.0
60,Kent,Drug offences,0.063,All offences,2.0
102,Kent,Fraud offences,0.056,All other<br>offences,0.0
144,Kent,Miscellaneous<br>crimes against<br>society,0.091,All other<br>offences,0.0
186,Kent,Possession of<br>weapons,0.007,All other<br>offences,0.0
228,Kent,Public order<br>offences,0.042,All other<br>offences,0.0
270,Kent,Robbery,0.021,All other<br>offences,0.0
312,Kent,Sexual offences,0.007,All other<br>offences,0.0
354,Kent,Summary motoring,0.014,All other<br>offences,0.0
396,Kent,Summary non-<br>motoring,0.077,All other<br>offences,0.0


In [165]:
pfa_df = pd.concat([pfa_df, 
                pd.DataFrame.from_records([{'pfa': pfa_df['pfa'].iloc[0], 
                                            'offence':"All other<br>offences", 
                                            'proportion': pfa_df.loc[~filt, 'proportion'].sum(), 
                                            'parent':"All offences",
                                            'plot_order': 0}])
                                            ],
                                            ignore_index=True
                ).sort_values(by=['plot_order'], ascending=True)
pfa_df

,pfa,offence,proportion,parent,plot_order
0,Kent,Criminal damage and<br>arson,0.014,All other<br>offences,0.0
2,Kent,Fraud offences,0.056,All other<br>offences,0.0
3,Kent,Miscellaneous<br>crimes against<br>society,0.091,All other<br>offences,0.0
4,Kent,Possession of<br>weapons,0.007,All other<br>offences,0.0
5,Kent,Public order<br>offences,0.042,All other<br>offences,0.0
6,Kent,Robbery,0.021,All other<br>offences,0.0
7,Kent,Sexual offences,0.007,All other<br>offences,0.0
8,Kent,Summary motoring,0.014,All other<br>offences,0.0
9,Kent,Summary non-<br>motoring,0.077,All other<br>offences,0.0
12,Kent,All other<br>offences,0.329,All offences,0.0


In [9]:
pfa_df = df2[df2["pfa"] == "Kent"]

#Creating an 'All other offences' parent value to contain offences listed in highlighted_offence_groups
pfa_df = pd.concat([pfa_df, 
                pd.DataFrame.from_records([{'pfa': pfa_df['pfa'].iloc[0], 
                                            'offence':"All other<br>offences", 
                                            'proportion': pfa_df.loc[~filt, 'proportion'].sum(), 
                                            'parent':"All offences",
                                            'plot_order': 0}])
                                            ],
                                            ignore_index=True
                ).sort_values(by=['plot_order', 'proportion'], ascending=True)
    

## Chart title
title = textwrap.wrap(f'<b>Imprisonment of women in {pfa_df["pfa"].iloc[0]} by offence group in 2022</b>', width=100)

fig = go.Figure()

fig.add_trace(go.Sunburst(
    labels=pfa_df['offence'],
    parents=pfa_df['parent'],
    values=pfa_df['proportion'],
    sort=False,
    branchvalues='total',
    texttemplate="%{label} <b>%{percentRoot: .0%}</b>",
    hovertemplate="<b>%{label}</b><br>%{percentParent: .0%} of %{parent}<extra></extra>",
    hoverinfo='label+percent parent',
    insidetextorientation='radial',
    rotation=300,
)
            )

fig.update_layout(
margin = dict(t=0, l=0, r=0, b=0),
# margin = dict(t=75, l=20, r=20, b=50),
title="<br>".join(title),
title_y=0.94,
title_yanchor="bottom",
uniformtext_minsize=8,
uniformtext_mode='hide',
width=800,
height=800,
)

## Chart annotations
annotations = []
# Adding source label
source = prt_theme.sourceAnnotation("Ministry of Justice, Criminal justice statistics", annotations)

# Adding annotations to layout
fig.update_layout(annotations=annotations)

fig.show()

# export_path = Path.joinpath(Path.cwd(), f"{config['data']['outPath']}", "custody_offences_2022")
# export_path.mkdir(parents=True, exist_ok=True) #generate if does not exist

# Setting filename variable and full path

# filename = str(pfa_df["pfa"].iloc[0]) 
# export_eps_path = Path.joinpath(export_path, f'{filename}' + '.eps')

# fig.write_image(export_eps_path)

In [176]:
for pfa in df2['pfa'].unique():
    pfa_df = df2[df2["pfa"] == pfa]

    #Creating an 'All other offences' parent value to contain offences listed in highlighted_offence_groups
    pfa_df = pd.concat([pfa_df, 
                    pd.DataFrame.from_records([{'pfa': pfa_df['pfa'].iloc[0], 
                                                'offence':"All other<br>offences", 
                                                'proportion': pfa_df.loc[~filt, 'proportion'].sum(), 
                                                'parent':"All offences",
                                                'plot_order': 0}])
                                                ],
                                                ignore_index=True
                    ).sort_values(by=['plot_order', 'proportion'], ascending=True)
        

    ## Chart title
    title = textwrap.wrap(f'<b>Imprisonment of women in {pfa_df["pfa"].iloc[0]} by offence group in 2022</b>', width=100)

    fig = go.Figure()

    fig.add_trace(go.Sunburst(
        labels=pfa_df['offence'],
        parents=pfa_df['parent'],
        values=pfa_df['proportion'],
        sort=False,
        branchvalues='total',
        texttemplate="%{label} <b>%{percentRoot: .0%}</b>",
        hovertemplate="<b>%{label}</b><br>%{percentParent: .0%} of %{parent}<extra></extra>",
        hoverinfo='label+percent parent',
        insidetextorientation='radial',
        rotation=300,
    )
                )

    fig.update_layout(
    margin = dict(t=75, l=0, r=0, b=0),
    # margin = dict(t=75, l=20, r=20, b=50),
    title="<br>".join(title),
    title_y=0.94,
    title_yanchor="bottom",
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    width=800,
    height=800,
    )

    ## Chart annotations
    annotations = []
    # Adding source label
    source = prt_theme.sourceAnnotation("Ministry of Justice, Criminal justice statistics", annotations)

    # Adding annotations to layout
    fig.update_layout(annotations=annotations)

    fig.show()

    # export_path = Path.joinpath(Path.cwd(), f"{config['data']['outPath']}", "custody_offences_2022")
    # export_path.mkdir(parents=True, exist_ok=True) #generate if does not exist

    # Setting filename variable and full path

    # filename = str(pfa_df["pfa"].iloc[0]) 
    # export_eps_path = Path.joinpath(export_path, f'{filename}' + '.eps')

    # fig.write_image(export_eps_path)

In [ ]:
pfa_df

In [ ]:
for pfa in df2['pfa'].unique():
    pfa_df = df2[df2["pfa"] == pfa]
    
    #Creating an 'All other offences' parent value to contain offences listed in highlighted_offence_groups
    pfa_df = pd.concat([pfa_df, 
                    pd.DataFrame.from_records([{'pfa': pfa_df['pfa'].iloc[0], 
                                                'offence':"All other<br>offences", 
                                                'proportion': pfa_df.loc[~filt, 'proportion'].sum(), 
                                                'parent':"All offences"}])
                                                ],
                                                ignore_index=True
                    ).sort_values(by=['offence'], ascending=False)
        
        
    ## Chart title
    title = textwrap.wrap(f'<b>Imprisonment of women in {pfa_df["pfa"].iloc[0]}<br>by offence group in 2022</b>', width=100)

    fig = go.Figure()

    fig.add_trace(go.Sunburst(
        labels=pfa_df['offence'],
        parents=pfa_df['parent'],
        values=pfa_df['proportion'],
        sort=False,
        branchvalues='total',
        texttemplate="%{label}<br>%{percentRoot: .0%}",
        hovertemplate="<b>%{label}</b><br>%{percentParent: .0%} of %{parent}<extra></extra>",
        hoverinfo='label+percent parent',
        insidetextorientation='horizontal',
        rotation=90,
        domain=dict(column=0),
    )
            )
    
    fig.add_trace(go.Sunburst(
        labels=pfa_df['offence'],
        parents=pfa_df['parent'],
        values=pfa_df['proportion'],
        sort=True,
        branchvalues='total',
        texttemplate="%{label}<br>%{percentRoot: .0%}",
        hovertemplate="<b>%{label}</b><br>%{percentParent: .0%} of %{parent}<extra></extra>",
        hoverinfo='label+percent parent',
        insidetextorientation='horizontal',
        rotation=90,
        level='All other<br>offences',
        domain=dict(column=1)
    )
                )
    
    fig.update_layout(
    margin = dict(t=75, l=0, r=20, b=50),
    grid= dict(columns=2, rows=1),
    title="<br>".join(title),
    title_y=0.94,
    title_yanchor="bottom",
    uniformtext_minsize=7, 
    uniformtext_mode='hide',
    width=800,
    height=500,    
    )

    ## Chart annotations
    annotations = []
    # Adding source label
    source = prt_theme.sourceAnnotation("Ministry of Justice, Criminal justice statistics", annotations)

    # Adding annotations to layout
    fig.update_layout(annotations=annotations)

    fig.show()

    fig.update_layout(uniformtext_mode='show')
    fig.show()

    # export_path = Path.joinpath(Path.cwd(), f"{config['data']['outPath']}", "custody_offences_2022")
    # export_path.mkdir(parents=True, exist_ok=True) #generate if does not exist

    # Setting filename variable and full path

    # filename = str(pfa_df["pfa"].iloc[0])
    # export_eps_path = Path.joinpath(export_path, f'{filename}' + '.eps')

    # fig.write_image(export_eps_path)

In [ ]:
print(fig)

Attempting a different graph style

In [ ]:
pfa_df = df2[df2["pfa"] == "Kent"]

#Creating an 'All other offences' parent value to contain offences listed in highlighted_offence_groups
pfa_df = pd.concat([pfa_df, 
                pd.DataFrame.from_records([{'pfa': pfa_df['pfa'].iloc[0], 
                                            'offence':"All other<br>offences", 
                                            'proportion': pfa_df.loc[~filt, 'proportion'].sum(), 
                                            'parent':"All offences"}])
                                            ],
                                            ignore_index=True
                ).sort_values(by=['offence'], ascending=False)
    
    
## Chart title
title = textwrap.wrap(f'<b>Imprisonment of women in {pfa_df["pfa"].iloc[0]}<br>by offence group in 2022</b>', width=100)

fig =go.Figure(go.Icicle(
    labels=pfa_df['offence'],
    parents=pfa_df['parent'],
    values=pfa_df['proportion'],
    root_color="lightgrey"
))

fig.show()

Let's loop back up and remove the breaks from the text.

In [ ]:
#Melting df2 from wide to long
df2 = pd.melt(df, id_vars='pfa', value_vars=df.columns[1:], var_name='offence', value_name='proportion')

#Selecting the offences that I want to continue to display at the root of the sunburst diagram
highlighted_offence_groups = ['Theft offences', 'Drug offences', 'Violence against the person']
filt = df2['offence'].isin(highlighted_offence_groups)

df2.loc[filt, 'parent'] = "All offences" # This method prevents that annoying copy/view warning
df2.loc[~filt, 'parent'] = "All other offences"

In [ ]:
pfa_df = df2[df2["pfa"] == "Kent"]

#Creating an 'All other offences' parent value to contain offences listed in highlighted_offence_groups
pfa_df = pd.concat([pfa_df, 
                pd.DataFrame.from_records([{'pfa': pfa_df['pfa'].iloc[0], 
                                            'offence':"All other offences", 
                                            'proportion': pfa_df.loc[~filt, 'proportion'].sum(), 
                                            'parent':"All offences"}])
                                            ],
                                            ignore_index=True
                ).sort_values(by=['offence'], ascending=False)
    
    
## Chart title
title = textwrap.wrap(f'<b>Imprisonment of women in {pfa_df["pfa"].iloc[0]}<br>by offence group in 2022</b>', width=100)

fig = go.Figure()

fig.add_trace(go.Icicle(
    labels=pfa_df['offence'],
    parents=pfa_df['parent'],
    values=pfa_df['proportion'],
    root_color="lightgrey",
    branchvalues='total',
    # sort=False,
    texttemplate="%{label}%{percentRoot: .0%}",
    hovertemplate="<b>%{label}</b><br>%{percentParent: .0%} of %{parent}<extra></extra>",
    )
    )

fig.update_layout(
margin = dict(t=85, l=20, r=20, b=40),
title="<br>".join(title),
title_y=0.94,
title_yanchor="bottom",
uniformtext_minsize=9, 
uniformtext_mode='show',
width=500,
height=800,    
)

## Chart annotations
annotations = []
# Adding source label
source = prt_theme.sourceAnnotation("Ministry of Justice, Criminal justice statistics", annotations)

# Adding annotations to layout
fig.update_layout(annotations=annotations)

fig.show()

In [ ]:
pfa_df = df2[df2["pfa"] == "Kent"]

#Creating an 'All other offences' parent value to contain offences listed in highlighted_offence_groups
pfa_df = pd.concat([pfa_df, 
                pd.DataFrame.from_records([{'pfa': pfa_df['pfa'].iloc[0], 
                                            'offence':"All other offences", 
                                            'proportion': pfa_df.loc[~filt, 'proportion'].sum(), 
                                            'parent':"All offences"}])
                                            ],
                                            ignore_index=True
                ).sort_values(by=['offence'], ascending=False)
    
    
## Chart title
title = textwrap.wrap(f'<b>Imprisonment of women in {pfa_df["pfa"].iloc[0]}<br>by offence group in 2022</b>', width=100)

fig = go.Figure()

mask = pfa_df['parent'] != "All other offences"

fig.add_trace(go.Icicle(
    labels=pfa_df['offence'][mask],
    parents=pfa_df['parent'][mask],
    values=pfa_df['proportion'][mask],
    root_color="lightgrey",
    branchvalues='total',
    # sort=False,
    texttemplate="%{label}%{percentRoot: .0%}",
    hovertemplate="<b>%{label}</b><br>%{percentParent: .0%} of %{parent}<extra></extra>",
    domain=dict(column=0),
    )
    )


fig.add_trace(go.Icicle(
    labels=pfa_df['offence'],
    parents=pfa_df['parent'],
    values=pfa_df['proportion'],
    root_color="lightgrey",
    branchvalues='total',
    sort=True,
    texttemplate="%{label}%{percentRoot: .0%}",
    hovertemplate="<b>%{label}</b><br>%{percentParent: .0%} of %{parent}<extra></extra>",
    level='All other offences',
    domain=dict(column=1),
    )
    )

fig.update_layout(
margin = dict(t=100, l=20, r=20, b=40),
grid= dict(columns=2, rows=1),
title="<br>".join(title),
title_y=0.94,
title_yanchor="bottom",
uniformtext_minsize=11, 
uniformtext_mode='show',
width=800,
height=800,    
)

## Chart annotations
annotations = []
# Adding source label
source = prt_theme.sourceAnnotation("Ministry of Justice, Criminal justice statistics", annotations)

# Adding annotations to layout
fig.update_layout(annotations=annotations)

fig.show()

In [ ]:
mask = pfa_df['parent'] != "All other offences"
pfa_df[mask]

In [ ]:
for pfa in df2['pfa'].unique():
    pfa_df = df2[df2["pfa"] == pfa]

    pfa_df = df2[df2["pfa"] == pfa]

    #Creating an 'All other offences' parent value to contain offences listed in highlighted_offence_groups
    pfa_df = pd.concat([pfa_df, 
                    pd.DataFrame.from_records([{'pfa': pfa_df['pfa'].iloc[0], 
                                                'offence':"All other offences", 
                                                'proportion': pfa_df.loc[~filt, 'proportion'].sum(), 
                                                'parent':"All offences"}])
                                                ],
                                                ignore_index=True
                    ).sort_values(by=['offence'], ascending=False)
        
        
    ## Chart title
    title = textwrap.wrap(f'<b>Imprisonment of women in {pfa_df["pfa"].iloc[0]}<br>by offence group in 2022</b>', width=100)

    fig =go.Figure(go.Icicle(
        labels=pfa_df['offence'],
        parents=pfa_df['parent'],
        values=pfa_df['proportion'],
        root_color="lightgrey",
        branchvalues='total',
        sort=True,
        # texttemplate="%{label}<br>%{percentRoot: .0%}",
        texttemplate="%{label}%{percentRoot: .0%}",
        hovertemplate="<b>%{label}</b><br>%{percentParent: .0%} of %{parent}<extra></extra>",
    ))

    fig.update_layout(
    margin = dict(t=75, l=20, r=20, b=50),
    title="<br>".join(title),
    title_y=0.94,
    title_yanchor="bottom",
    uniformtext_minsize=7, 
    uniformtext_mode='show',
    width=800,
    height=500,    
    )

    ## Chart annotations
    annotations = []
    # Adding source label
    source = prt_theme.sourceAnnotation("Ministry of Justice, Criminal justice statistics", annotations)

    # Adding annotations to layout
    fig.update_layout(annotations=annotations)

    fig.show()

In [ ]:
pfa_df = df2[df2["pfa"] == "Kent"]

#Creating an 'All other offences' parent value to contain offences listed in highlighted_offence_groups
pfa_df = pd.concat([pfa_df, 
                pd.DataFrame.from_records([{'pfa': pfa_df['pfa'].iloc[0], 
                                            'offence':"All other<br>offences", 
                                            'proportion': pfa_df.loc[~filt, 'proportion'].sum(), 
                                            'parent':"All offences"}])
                                            ],
                                            ignore_index=True
                ).sort_values(by=['offence'], ascending=False)
    
    
## Chart title
title = textwrap.wrap(f'<b>Imprisonment of women in {pfa_df["pfa"].iloc[0]}<br>by offence group in 2022</b>', width=100)

fig = go.Figure()

fig.add_trace(go.Sunburst(
    labels=pfa_df['offence'],
    parents=pfa_df['parent'],
    values=pfa_df['proportion'],
    sort=False,
    branchvalues='total',
    texttemplate="%{label}<br>%{percentRoot: .0%}",
    hovertemplate="<b>%{label}</b><br>%{percentParent: .0%} of %{parent}<extra></extra>",
    hoverinfo='label+percent parent',
    insidetextorientation='horizontal',
    rotation=90,
    domain=dict(column=0),
)
        )